This Notebook addresse the cleaning and merging of several CSV files to produce a final Data Frame for plotting. We located our data sets from CDC Websites, Department of Labor Statistics, Data World, and Keggle.All original data sets are in teh Raw CSV Directory and all cleaned data frames are saved as CSV in the Cleaned_Data Directory.

In [1]:
# dependencies
import pandas as pd

In [2]:
#Open and Clean Moderan CSV
# open next CSV file for merge with cleaned file
moderna_csv = 'Raw Csv/COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Moderna.csv'
# read csv
moderna = pd.read_csv(moderna_csv)
moderna['Jurisdiction'] = moderna.Jurisdiction.str.replace(r'[^a-zA-Z ]\s?',r'',regex=True)
# rename Jurisdiction column State for merge
moderna = moderna.rename(columns={"Jurisdiction": "State",
                                   'Total Moderna Allocation "First Dose" Shipments': 'Total Two Dose Regimen Moderna'})
# removing remaining unneeded columns
moderna = moderna.drop(columns=['HHS Region',
                                            'Doses allocated week of 12/21',
                                            'Second Dose Shipment (28 days later) week of 12/21',
                                            'Doses allocated week of 12/28',
                                            'Second Dose Shipment (28 days later) week of 12/28',
                                            'Doses allocated for distribution week of 01/04',
                                            'Second dose shipment for distribution (28 days later) week of 01/04',
                                            'Doses allocated for distribution week of 01/10',
                                            'Second dose shipment for distribution (28 days later) week of 01/10',
                                            'Doses allocated for distribution week of 01/18',
                                            'Second dose shipment for distribution (28 days later) week of 01/18',
                                            'Doses allocated for distribution week of 01/25',
                                            'Second dose shipment for distribution (28 days later) week of 01/25',
                                            'Doses allocated for distribution week of 02/01',
                                            'Second dose shipment for distribution (28 days later) week of 02/01',
                                            'Total Allocation Moderna"Second Dose" Shipments'])
#convert doses to int for later
# remove commas
moderna['Total Two Dose Regimen Moderna'] = moderna['Total Two Dose Regimen Moderna'].str.replace(",", "")
# replace nan with 0
moderna['Total Two Dose Regimen Moderna'] = moderna['Total Two Dose Regimen Moderna'].fillna(0)
# convert column to int
moderna['Total Two Dose Regimen Moderna'] = moderna['Total Two Dose Regimen Moderna'].astype(int)


#save cleaned csv for later merge
moderna.to_csv("Cleaned_Data/Modern_Cleaned.csv", index=False)
moderna.head()

,State,Total Two Dose Regimen Moderna
0,Connecticut,205200
1,Maine,78500
2,Massachusetts,395100
3,New Hampshire,78600
4,Rhode Island,61700


In [3]:
#open and clean pfizer csv

# open next CSV file for merge with cleaned file
pfizer_csv = 'Raw CSV/COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Pfizer.csv'
# read csv
pfizer = pd.read_csv(pfizer_csv)
pfizer['Jurisdiction'] = pfizer.Jurisdiction.str.replace(r'[^a-zA-Z ]\s?',r'',regex=True)
# rename Jurisdiction column State for merge
pfizer = pfizer.rename(columns={"Jurisdiction": "State",
                                 'Total Pfizer Allocation "First Dose" Shipments': 'Total "Two Dose Regimen" Pfizer',})
# removing remaining unneeded columns
# removing remaining unneeded columns
pfizer = pfizer.drop(columns=['HHS Region',
                                'Doses allocated week of 12/14',
                                'Second Dose Shipment (21 days later) week of 12/14',
                                'Doses allocated week of 12/21',
                                'Second Dose Shipment (21 days later) week of 12/21',
                                'Doses allocated week of 12/28',
                                'Second Dose Shipment (21 days later) week of 12/28',
                                'Doses allocated for distribution week of 01/04',
                                'Second dose shipment for distribution (21 days later) week of 01/04',
                                'Doses allocated for distribution week of 01/10',
                                'Second dose shipment for distribution (21 days later) week of 01/10',
                                'Doses allocated for distribution week of 01/18',
                                'Second dose shipment for distribution (21 days later) week of 01/18',
                                'Doses allocated for distribution week of 01/25',
                                'Second dose shipment for distribution (21 days later) week of 01/25',
                                'Total Allocation Pfizer "Second Dose" Shipments'])
#convert doses to int for later
# remove commas
pfizer['Total "Two Dose Regimen" Pfizer'] = pfizer['Total "Two Dose Regimen" Pfizer'].str.replace(",", "")
# replace nan with 0
pfizer['Total "Two Dose Regimen" Pfizer'] = pfizer['Total "Two Dose Regimen" Pfizer'].fillna(0)
# convert column to int
pfizer['Total "Two Dose Regimen" Pfizer'] = pfizer['Total "Two Dose Regimen" Pfizer'].astype(int)
#save cleaned csv for later merge
pfizer.to_csv("Cleaned_Data/Pfizer_Cleaned.csv", index=False)
#pfizer.head()

In [4]:
#merge vacc dataframes
vaccine_distribuiton_merged = pd.merge(moderna, pfizer)
#save merged data fram for later
vaccine_distribuiton_merged.to_csv("Cleaned_Data/Pfizer_Moderna_Merged.csv", index=False)
vaccine_distribuiton_merged.head()

,State,Total Two Dose Regimen Moderna,"Total ""Two Dose Regimen"" Pfizer"
0,Connecticut,205200,173550
1,Maine,78500,67275
2,Massachusetts,395100,330525
3,New Hampshire,78600,67275
4,Rhode Island,61700,52650


In [5]:
#open and clean Provisional Stats CSV
# file path
provisional_stats = 'Raw Csv/Provisional_COVID-19_Death_Counts_by_Sex__Age__and_State.csv'
#read the data file
ps_df = pd.read_csv(provisional_stats)
# drop unneeded columns
ps_df = ps_df.drop(columns=['Data as of', 'Start Date', 'End Date', 'Footnote'])
# group total stats based on state totals
# obtain state total rows
ps_df_filtered = ps_df[ps_df['Sex']=='All Sexes']
# drilling to state totals
ps_df_filtered = ps_df_filtered[ps_df_filtered['Age group'] == 'All Ages']
# dropping US Totals row
indexname= ps_df_filtered[ps_df_filtered['State']=='United States'].index
ps_df_filtered.drop(indexname, inplace =True)
# removing remaining unneeded columns
ps_df_filtered = ps_df_filtered.drop(columns=['Sex', 'Age group'])

# reset index
ps_df_filtered = ps_df_filtered.reset_index(drop=True)
#convert columns to int for later use
# remove commas
ps_df_filtered['COVID-19 Deaths'] = ps_df_filtered['COVID-19 Deaths'].str.replace(",", "")

#replace NaN with 0
ps_df_filtered = ps_df_filtered.fillna(0)

# convert column to int
pfizer['Total "Two Dose Regimen" Pfizer'] = pfizer['Total "Two Dose Regimen" Pfizer'].astype(int)
ps_df_filtered['COVID-19 Deaths'] = ps_df_filtered['COVID-19 Deaths'].astype(int)

#drop unneedd columns
ps_df_filtered = ps_df_filtered.drop(columns = ['Total Deaths', 'Pneumonia Deaths', 'Pneumonia and COVID-19 Deaths',
                                               'Influenza Deaths','Pneumonia, Influenza, or COVID-19 Deaths', 'COVID-19 Deaths' ])

# view new head
ps_df_filtered.head()
#save cleaned csv for later merge
ps_df_filtered.to_csv("Cleaned_Data/Cleaned_Provisional_Stats.csv", index=False)
# check # of rows
#len(ps_df_filtered)
#print df header
ps_df.head()

,State,Sex,Age group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths"
0,United States,All Sexes,All Ages,"347,131","3,329,907","331,354","161,474","8,726","524,629"
1,United States,All Sexes,Under 1 year,39,"18,436",193,6,21,247
2,United States,All Sexes,0-17 years,165,"32,453",536,29,175,847
3,United States,All Sexes,1-4 years,21,"3,388",120,3,59,197
4,United States,All Sexes,5-14 years,58,"5,440",153,11,74,274


In [6]:
#open population density csv
# open next CSV file for merge with cleaned file
popden = 'Raw CSV/Population_Density.csv'
# read csv
population_density = pd.read_csv(popden)
#print head
population_density.head()
# check # of rows
#len(population_density)

,State,Density,Pop,LandArea
0,New Jersey,1206.7609,8874520,7354
1,Rhode Island,1026.6054,1061510,1034
2,Massachusetts,886.1846,6912240,7800
3,Connecticut,733.7505,3552820,4842
4,Maryland,624.8522,6065440,9707


In [7]:
#merge df merge how right to drop NYC and Puerto Rico due to no density info
stats_popden_merge = pd.merge(ps_df_filtered, population_density, on='State', how ='right')
# display new df header
stats_popden_merge.head()

,State,Density,Pop,LandArea
0,Alabama,97.4270,4934190,50645
1,Alaska,1.2694,724357,570641
2,Arizona,66.2016,7520100,113594
3,Arkansas,58.3059,3033950,52035
4,California,254.2929,39613500,155779


In [8]:
#merge provisional stats/pop den with pfize/modenra-merge left to drop incomplete rows from vacc dist files
stats_den_pfi_mod = pd.merge(stats_popden_merge, vaccine_distribuiton_merged, on="State", how="left" )
#save cleaned csv for later merge
stats_den_pfi_mod.to_csv("Cleaned_Data/All_CSV_Combined.csv", index=False)
stats_den_pfi_mod.head()

,State,Density,Pop,LandArea,Total Two Dose Regimen Moderna,"Total ""Two Dose Regimen"" Pfizer"
0,Alabama,97.4270,4934190,50645,273100,227175
1,Alaska,1.2694,724357,570641,84700,62400
2,Arizona,66.2016,7520100,113594,387100,323700
3,Arkansas,58.3059,3033950,52035,165800,141375
4,California,254.2929,39613500,155779,2178600,1806675


In [9]:
#Open and Clean cases by state CSV
# open next CSV file for merge with cleaned file
cases_by_state_csv = 'Raw CSV/Confirmed_Cases_By_State.csv'
# read csv
cases_by_state_df = pd.read_csv(cases_by_state_csv)
#rename columns
pfizer = pfizer.rename(columns={"Confirmed": "Confirmed Cases",
                                 'Deaths': 'Total Deaths',})
#convert doses to int for later
# remove commas
cases_by_state_df['Confirmed'] = cases_by_state_df['Confirmed'].str.replace(",", "")
cases_by_state_df['Deaths'] = cases_by_state_df['Deaths'].str.replace(",", "")
# replace nan with 0
cases_by_state_df['Confirmed'] = cases_by_state_df['Confirmed'].fillna(0)
cases_by_state_df['Deaths'] = cases_by_state_df['Deaths'].fillna(0)
# convert column to int
cases_by_state_df['Confirmed'] = cases_by_state_df['Confirmed'].astype(int)
cases_by_state_df['Deaths'] = cases_by_state_df['Deaths'].astype(int)

cases_by_state_df.head()

,State,Confirmed,Deaths
0,Alabama,449086,7172
1,Alaska,51603,250
2,Arizona,738561,12643
3,Arkansas,288964,4742
4,California,3169935,38224


In [15]:
#merge df with state total case counts
final_df = pd.merge(stats_den_pfi_mod, cases_by_state_df, on="State", how="left" )
final_df['Combined Total Doses'] = final_df['Total Two Dose Regimen Moderna'] + final_df['Total "Two Dose Regimen" Pfizer']

final_df = final_df.rename(columns={'Pop' : 'Population',
                                   'Total "Two Dose Regimen" Pfizer' : 'Total Two Dose Regimen Pfizer',
                                   'Confirmed' : 'Confirmed Cases',
                                   'Density' : 'Population Density',
                                   'Deaths' : 'COVID Deaths'})


#reindex columns
final_df = final_df[['State', 'Confirmed Cases', 'COVID Deaths', 'Population Density', 'Population', 'LandArea',
                    'Total Two Dose Regimen Moderna', 'Total Two Dose Regimen Pfizer', 'Combined Total Doses']]

#save cleaned csv for later merge
final_df.to_csv("Cleaned_Data/Final.csv", index=False)
final_df.head()

,State,Confirmed Cases,COVID Deaths,Population Density,Population,LandArea,Total Two Dose Regimen Moderna,Total Two Dose Regimen Pfizer,Combined Total Doses
0,Alabama,449086,7172,97.4270,4934190,50645,273100,227175,500275
1,Alaska,51603,250,1.2694,724357,570641,84700,62400,147100
2,Arizona,738561,12643,66.2016,7520100,113594,387100,323700,710800
3,Arkansas,288964,4742,58.3059,3033950,52035,165800,141375,307175
4,California,3169935,38224,254.2929,39613500,155779,2178600,1806675,3985275
